GENERATE  METADATA, MFCC and GFCC FOR AUDIO DATASET 

In [12]:
import pandas as pd
from pathlib import Path
import glob 


# Read metadata file
metadata_file = "/home/conex/scream_detection/dataset/scream_detection.csv"

proc_class_names = ["cough"]

# Metadata file
df_new = pd.DataFrame(columns=['filepath', 'class_id','class_name'])

# gen metadata for human noises
dataset_paths = "/home/conex/scream_detection/dataset/vs_release_44k/data_44k/cough/*.wav"
audio_files = glob.glob(dataset_paths)
idx = 0
for file in audio_files:
    filepath = file 
    class_name = ((filepath.split("/")[-1]).split(".")[0]).split("_")[-1]
    class_id = -1
    if(class_name == "laughter"):
        class_id = 0
    elif(class_name == "sigh"):
        class_id = 1
    elif(class_name == "cough"):
        class_id = 2
    elif(class_name == "throatclearing"):
        class_id = 3
    elif(class_name == "sneeze"):
        class_id = 4
    elif(class_name == "sniff"):
        class_id = 5

    if(class_name in proc_class_names):
        df_new.loc[idx] = [filepath]+[str(class_id)]+[class_name]
        idx+=1
    

# gen enviormental noise metadata
idx = 0
env_dataset_path = "/home/conex/scream_detection/dataset/chime-home/*.wav"
env_audio_file = glob.glob(env_dataset_path)
for file in env_audio_file:
    filepath = file 
    class_name = "env"
    class_id = 6

    df_new.loc[idx] = [filepath]+[str(class_id)]+[class_name]
    idx+=1
    if(idx==2000):
        break


df_new.to_csv(metadata_file, sep=',', encoding='utf-8')


GENERATE MFFC AND GFCC FEATURE SETS

In [2]:
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
from spafe.features.gfcc import gfcc

%matplotlib inline

metadata_file = "/home/conex/scream_detection/dataset/scream_detection.csv"
dataset = pd.read_csv(metadata_file)
dataset.head()

num_feature_points_mfcc = 13
num_feature_points_gfcc = 13

'''
plt.ion()
fig = plt.figure()
'''

for i in range(len(dataset)):
    file_error = 0

    file_path = (dataset.loc[i]['filepath'])
    audio_data, sample_rate = librosa.load(file_path,sr=48000)

    # Gen MFCC and GFCC  
    try:
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=num_feature_points_mfcc)
        gfccs = gfcc(audio_data, fs=sample_rate)
    except IndexError as e:
        print(str(i)+" : " + file_path.split("/")[-1]+" : "+str(e))
        file_error = 1
    except Exception as e:
        print(str(i)+" : " + file_path.split("/")[-1]+" : "+str(e))
        file_error = 1

    # Write features
    if(file_error == 0):    
        print("Processed : " + file_path.split("/")[-1])
        file_name_parts = file_path.split(".")[0]
        if(len(file_name_parts) == 2 ):
            npz_file_path_base = file_path.split(".")[0]
        else:
            npz_file_path_base = file_path.split(".")[0]+file_path.split(".")[1]
        mfcc_file_name = npz_file_path_base+"_mfcc.npz"
        gfcc_file_name = npz_file_path_base+"_gfcc.npz"
        np.savez_compressed(mfcc_file_name,a=np.asarray(mfccs))
        np.savez_compressed(gfcc_file_name,a=gfccs)

    
    
    #--------------------------------------------------------------------------------------------------#
    # Plots
    '''

    mfccs_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, 
                                       n_mels=128,fmax=8000)
                                       
    fig.clf()
    
    ax_1 = fig.add_subplot(211)
    img = librosa.display.specshow(librosa.power_to_db(mfccs_spec, ref=np.max),
                                x_axis='time', y_axis='mel', fmax=8000,
                                ax=ax_1)
    fig.colorbar(img, ax=[ax_1])
    ax_1.set(title='Mel spectrogram')
    ax_1.label_outer()

    ax_2 = fig.add_subplot(212)
    img = librosa.display.specshow(mfccs, x_axis='time', ax=ax_2)
    fig.colorbar(img, ax=[ax_2])
    ax_2.set(title='MFCC')



    plt.draw()
    plt.pause(0.1)
    '''
    #--------------------------------------------------------------------------------------------------#
    

Processed : CR_lounge_270110_1632.s3300_chunk10.48kHz.wav
Processed : CR_lounge_230110_1501.s2700_chunk55.48kHz.wav
Processed : CR_lounge_220110_0731.s3600_chunk32.48kHz.wav
Processed : CR_lounge_200110_1601.s300_chunk60.48kHz.wav
Processed : CR_lounge_230110_1036.s0_chunk27.48kHz.wav
Processed : CR_lounge_210110_0739.s1200_chunk31.48kHz.wav
Processed : CR_lounge_210110_0739.s300_chunk47.48kHz.wav
Processed : CR_lounge_220110_0731.s1500_chunk40.48kHz.wav
Processed : CR_lounge_200110_1601.s1200_chunk60.48kHz.wav
Processed : CR_lounge_200110_1601.s600_chunk19.48kHz.wav
Processed : CR_lounge_230110_1501.s3600_chunk44.48kHz.wav
Processed : CR_lounge_230110_1036.s1500_chunk39.48kHz.wav
Processed : CR_lounge_230110_1036.s600_chunk68.48kHz.wav
Processed : CR_lounge_270110_1632.s2100_chunk64.48kHz.wav
Processed : CR_lounge_210110_0739.s2100_chunk7.48kHz.wav
Processed : CR_lounge_200110_1711.s300_chunk31.48kHz.wav
Processed : CR_lounge_200110_1601.s1500_chunk20.48kHz.wav
Processed : CR_lounge_2

/home/conex/miniconda3/envs/comm/lib/python3.9/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Processed : m2447_0_cough.wav
Processed : m1876_0_cough.wav
Processed : m1449_0_cough.wav
Processed : f1849_0_cough.wav
Processed : m3136_0_cough.wav
Processed : m2923_0_cough.wav
Processed : f2904_0_cough.wav
Processed : f3299_0_cough.wav
Processed : m1892_0_cough.wav
Processed : m1520_0_cough.wav
Processed : m1129_0_cough.wav
Processed : m1707_0_cough.wav
Processed : m2340_0_cough.wav
Processed : f2344_0_cough.wav
Processed : m2637_0_cough.wav
Processed : f2257_0_cough.wav
Processed : m0328_0_cough.wav
Processed : f1650_0_cough.wav
Processed : m2634_0_cough.wav
Processed : m0526_0_cough.wav
Processed : m0001_0_cough.wav
Processed : f2319_0_cough.wav
Processed : m2124_0_cough.wav
Processed : m2131_0_cough.wav
Processed : f2745_0_cough.wav
Processed : m3006_0_cough.wav
Processed : f0314_0_cough.wav
Processed : m2982_0_cough.wav
Processed : m2645_0_cough.wav
Processed : m0608_0_cough.wav
Processed : f0622_0_cough.wav
Processed : m0389_0_cough.wav
Processed : m1231_0_cough.wav
Processed 

TRAIN RANDOM FOREST CLASSIFIER MODEL

In [3]:
import pandas as pd
from imblearn.over_sampling import ADASYN
import numpy as np
import glob
import cv2

from sklearn import model_selection
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

dataset_paths = "/home/conex/scream_detection/dataset/vs_release_44k/data_44k/cough/*.npz"
env_dataset_path = "/home/conex/scream_detection/dataset/chime-home/*.npz"
mfcc_files = []
gfcc_files = []
pred_classes = []

data_feature_set_files = glob.glob(dataset_paths)
counter = 0
for filename in data_feature_set_files:
    if("mfcc" in  filename):
        mfcc_files.append(filename)
    elif("gfcc" in filename):
        gfcc_files.append(filename)
    if(counter%2 == 0):
        pred_classes.append("cough")
    counter+=1

env_feature_set_files = glob.glob(env_dataset_path)
counter = 0
for filename in env_feature_set_files:
    if("mfcc" in  filename):
        mfcc_files.append(filename)
    elif("gfcc" in filename):
        gfcc_files.append(filename)
    if(counter%2 == 0):
        pred_classes.append("env")
    counter+=1

pred_classes = np.asarray(pred_classes)
input_feature_set = []
window_sz = 2500
for idx in range(len(mfcc_files)):
    mfcc_data = (np.load(mfcc_files[idx])['a'])
    gfcc_data = (np.load(gfcc_files[idx])['a'])
    mfcc_data_resz = (cv2.resize(mfcc_data, dsize=(250,13),interpolation=cv2.INTER_CUBIC))
    gfcc_data_resz = (cv2.resize(gfcc_data, dsize=(13,150),interpolation=cv2.INTER_CUBIC))
    mfcc_data_resz = mfcc_data_resz.flatten()
    gfcc_data_resz = gfcc_data_resz.flatten()
    feature_set = np.concatenate((mfcc_data_resz, gfcc_data_resz))
    input_feature_set.append(feature_set)
    '''
    mfcc_data = (np.load(mfcc_files[idx])['a']).flatten()
    gfcc_data = (np.load(gfcc_files[idx])['a']).flatten()
    feature_set = np.concatenate((mfcc_data, gfcc_data))
    #inputs_m, inputs_s = feature_set.mean(), feature_set.std()
    #feature_set = (feature_set - inputs_m) / inputs_s
    feature_set = (cv2.resize(feature_set, dsize=(window_sz,1),
                              interpolation=cv2.INTER_CUBIC)).flatten()
    input_feature_set.append(feature_set)
    '''
row=len(input_feature_set)
column=len(input_feature_set[0])
print(f'Rows:{row}, Column:{column}')

row=len(pred_classes)
print(f'Rows:{row}')

# Split  data into train/test data sets , 30% testing and 70% train
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (input_feature_set, 
                                                                     pred_classes, 
                                                                     test_size=0.3,
                                                                     random_state=42)

# Resample dataset                                                                      
#adasyn = ADASYN(random_state=42)
#X_resampled, Y_resampled = adasyn.fit_resample(X_train, Y_train)

# Build an RandomForestClassifier model 
clf_rf = RandomForestClassifier(n_estimators=30, max_depth=4, 
                                max_features = 0.4, random_state=42,
                                verbose=100,n_jobs=4)
clf_rf.fit(np.asarray(X_train),(np.asarray(Y_train)).ravel())

#Check accuracy score
print(clf_rf.score(np.asarray(X_test), np.asarray(Y_test)))



# Validate robustness of above model using K-Fold Cross validation technique
scores_res = model_selection.cross_val_score(clf_rf, np.asarray(input_feature_set), 
                                             (np.asarray(pred_classes)).ravel(), 
                                             cv=5, verbose=100,n_jobs=4)

# Print the accuracy of each fold (i.e. 5 as above we asked cv 5)
print(scores_res)

# And the mean accuracy of all 5 folds.
print(scores_res.mean())

#Prediction 
#in_data_prediction = [values in posture_features format]
#p_res = clf_ob.predict(in_data_for_prediction)


Rows:7006, Column:5200
Rows:7006
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 2 of 30
building tree 1 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    5.0s

building tree 6 of 30
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    5.0s
building tree 7 of 30
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    5.1s
building tree 8 of 30
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    5.1s
building tree 9 of 30
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    9.4s
building tree 10 of 30[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   10.0s

building tree 11 of 30
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   10.1s
building tree 12 of 30
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   10.2s
building tree 13 of 30[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   14.6s

building tree 14 of 30[Parallel(n_jobs=4)]: Done

752.95s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
753.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
753.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
753.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
753.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


[CV] START .....................................................................
[CV] START .....................................................................
[CV] START .....................................................................
[CV] START .....................................................................
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 30building tree 2 of 30
building tree 3 of 30
building tree 4 of 30

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 30building tree 2 of 30

building tree 3 of 30
building tree 4 of 30
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 30building tree 2 of 30

building tree 3 of 30
building tree 4 of 30
buil

EXPORT MODEL

In [4]:
import pickle
from sklearn.metrics import confusion_matrix,classification_report

filename = 'cough_detection.sav'
pickle.dump(clf_rf, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

test_preds = loaded_model.predict(X_test)
confusion_matrix_graph = confusion_matrix(Y_test, test_preds)
classification_report_res = classification_report(Y_test, test_preds)
print('\n*Classification Report:\n', classification_report_res)
print('\n*Confusion Matrix Report:\n', confusion_matrix_graph)




[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:    0.0s
[Para

INFERENCE TEST

In [2]:
import pickle
import librosa
from spafe.features.gfcc import gfcc
import numpy as np
import cv2

filename = 'cough_detection.sav'

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.verbose = 0

# do inference
num_feature_points_mfcc = 13
num_feature_points_gfcc = 13
window_sz = 2500

test_data_file = "/home/conex/scream_detection/dataset/vs_release_44k/data_44k/cough/f0269_0_cough.wav"
audio_data, sample_rate = librosa.load(test_data_file,sr=48000)
# Gen MFCC and GFCC  
mfcc_data = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=num_feature_points_mfcc)
gfcc_data = gfcc(audio_data, fs=sample_rate)
mfcc_data_resz = (cv2.resize(mfcc_data, dsize=(250,13),interpolation=cv2.INTER_CUBIC))
gfcc_data_resz = (cv2.resize(gfcc_data, dsize=(13,150),interpolation=cv2.INTER_CUBIC))
mfcc_data_resz = mfcc_data_resz.flatten()
gfcc_data_resz = gfcc_data_resz.flatten()
feature_set = np.concatenate((mfcc_data_resz, gfcc_data_resz))
p_res = loaded_model.predict(np.asarray([feature_set]))
print("Detection result for : "+test_data_file.split("/")[-1]+" : "+str(p_res))
probs = loaded_model.predict_proba(np.asarray([feature_set]))
print("Probabilities of detection : ")
print(probs)

Detection result for : f0269_0_cough.wav : ['cough']
Probabilities of detection : 
[[0.99876422 0.00123578]]
